# Inicialización de los parámetros

Pytorch nos da la posibilidad de [inicializar](https://pytorch.org/docs/stable/nn.init.html) los parámetros de una red. Aunque es algo que no se suele modificar, y se suele dejar la manera en la que Pytorch lo hace poir defecto, vamos a ver un ejemplo de inicialización por el método [Xavier](https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)

Descargamos un dataset

In [1]:
from sklearn import datasets

cancer = datasets.load_breast_cancer()

In [2]:
import pandas as pd

cancer_df = pd.DataFrame(cancer['data'], columns=cancer['feature_names'])
cancer_df['type'] = cancer['target']
cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,type
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


Creamos el dataset

In [45]:
import torch

class CancerDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        cols = [col for col in dataframe.columns if col != 'target']
        self.parameters = torch.from_numpy(dataframe[cols].values).type(torch.float32)
        self.targets = torch.from_numpy(dataframe['type'].values).type(torch.float32)
        self.targets = self.targets.reshape((len(self.targets), 1))

    def __len__(self):
        return len(self.parameters)

    def __getitem__(self, idx):
        parameters = self.parameters[idx]
        target = self.targets[idx]
        return parameters, target

ds = CancerDataset(cancer_df)
train_ds, valid_ds = torch.utils.data.random_split(ds, [int(0.8*len(ds)), len(ds) - int(0.8*len(ds))], generator=torch.Generator().manual_seed(42))

Ahora el dataloader

In [46]:
from torch.utils.data import DataLoader

BS_train = 64
BS_val = 128 # Solo hay 114 datos de validación, por lo que no se puede dividir en batches

train_dl = DataLoader(train_ds, batch_size=BS_train, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=BS_val, shuffle=False)

Creamos la red neuronal que inicialzia los pesos mediante el método [Xavier](https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)

In [47]:
from torch import nn

class CancerNeuralNetwork(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_layers=[100, 50, 20]):
        super().__init__()
        self.network = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, hidden_layers[0]),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layers[0], hidden_layers[1]),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layers[1], hidden_layers[2]),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layers[2], num_outputs),
        )
        self.activation = torch.nn.Sigmoid()

        self.apply(self._init_weights)
    
    # Weights initialization using Xavier's method
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            nn.init.zeros_(module.bias)

    def forward(self, x):
        logits = self.network(x)
        probs = self.activation(logits)
        return logits, probs

num_inputs = 31
num_outputs = 1
model = CancerNeuralNetwork(num_inputs, num_outputs)

Llevamos la red a la GPU

In [48]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model.to(device)

Using cuda device


CancerNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=31, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=20, bias=True)
    (5): ReLU()
    (6): Linear(in_features=20, out_features=1, bias=True)
  )
  (activation): Sigmoid()
)

Creamos la función de entrenamiento

In [49]:
LR = 1e-3

loss_fn2 = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

In [50]:
num_prints = 4

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # X and y to device
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        logits, probs = model(X)
        loss = loss_fn2(logits, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % int(len(dataloader)/num_prints) == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def val_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()

    with torch.no_grad():
        for X, y in dataloader:
            # X and y to device
            X, y = X.to(device), y.to(device)
            
            logits, probs = model(X)
            test_loss += loss_fn2(logits, y).item()
            correct += (probs.round() == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Entrenamos

In [51]:
epochs = 14
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dl, model, loss_fn2, optimizer)
    val_loop(val_dl, model, loss_fn2)
print("Done!")

Epoch 1
-------------------------------
loss: 3.863001  [    0/  455]
loss: 10.362530  [  128/  455]
loss: 4.719738  [  256/  455]
loss: 0.781383  [  384/  455]
Test Error: 
 Accuracy: 59.6%, Avg loss: 3.963881 

Epoch 2
-------------------------------
loss: 3.213980  [    0/  455]
loss: 1.604453  [  128/  455]
loss: 0.415575  [  256/  455]
loss: 0.586057  [  384/  455]
Test Error: 
 Accuracy: 40.4%, Avg loss: 1.085941 

Epoch 3
-------------------------------
loss: 1.143599  [    0/  455]
loss: 0.597836  [  128/  455]
loss: 0.762338  [  256/  455]
loss: 1.031210  [  384/  455]
Test Error: 
 Accuracy: 55.3%, Avg loss: 0.590678 

Epoch 4
-------------------------------
loss: 0.596846  [    0/  455]
loss: 0.721222  [  128/  455]
loss: 0.755344  [  256/  455]
loss: 0.945605  [  384/  455]
Test Error: 
 Accuracy: 52.6%, Avg loss: 0.600504 

Epoch 5
-------------------------------
loss: 0.613941  [    0/  455]
loss: 0.403116  [  128/  455]
loss: 0.497373  [  256/  455]
loss: 0.491463  [  38